In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras import models, layers
from tqdm import tqdm
from PIL import Image

In [ ]:
!pip install -U tensorflow-addons
import tensorflow_addons as tfa
tqdm_callback = tfa.callbacks.TQDMProgressBar()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
tf.config.list_physical_devices("GPU")
# tf.debugging.set_log_device_placement(True)

In [ ]:
raw_data = pd.read_csv("../input/challenges-in-representation-learning-facial-expression-recognition-challenge/icml_face_data.csv")
emotions = {0: 'Angry', 1: 'Disgust', 2: 'Fear', 3: 'Happy', 4: 'Sad', 5: 'Surprise', 6: 'Neutral'}

In [ ]:
hot_emotions = pd.get_dummies(raw_data['emotion'])
clear_hot_emotions = pd.DataFrame({"hot_emotions":[]})
for inx in tqdm(range(hot_emotions.shape[0])):
    hot_em = [list(hot_emotions.loc[inx])]
    tmp = pd.DataFrame({'hot_emotions':hot_em})
    clear_hot_emotions = clear_hot_emotions.append(tmp, ignore_index=True)
raw_data = pd.concat([raw_data, clear_hot_emotions], axis=1).drop('emotion', axis=1)

In [ ]:
raw_data

In [ ]:
def parse_data(data):
    dt = np.zeros(shape=(len(data), 48, 48, 1))
    lb = np.array(list(map(list, data['hot_emotions'])))
    
    for i, row in tqdm(enumerate(data.index)):
        image = np.fromstring(data.loc[row, ' pixels'], dtype=int, sep=' ')
        image = np.reshape(image, (48, 48, 1))
        dt[i] = image
        
    return dt, lb

train_data, train_lab = parse_data(raw_data[raw_data[" Usage"] == "Training"])
test_data, test_lab = parse_data(raw_data[raw_data[" Usage"] == "PublicTest"])
val_data, val_lab = parse_data(raw_data[raw_data[" Usage"] == "PrivateTest"])

In [ ]:
train_data.shape, test_data.shape, val_data.shape

In [ ]:
train_lab.shape, test_lab.shape, val_lab.shape

In [ ]:
def top_layer(layout):
  inner_layer = layers.Conv2D(32, 3, strides=2, padding='same')(layout)
  inner_layer = layers.BatchNormalization()(inner_layer)
  inner_layer = layers.Activation('relu')(inner_layer)
  # ----------------
  inner_layer = layers.Conv2D(64, 3, padding='same')(inner_layer)
  inner_layer = layers.BatchNormalization()(inner_layer)
  inner_layer = layers.Activation('relu')(inner_layer)
  # ----------------
  previous_block_activation = inner_layer
  for size in [128, 256, 728]:
    inner_layer = layers.Activation('relu')(inner_layer)
    inner_layer = layers.SeparableConv2D(size, 3, padding='same')(inner_layer)
    inner_layer = layers.BatchNormalization()(inner_layer)
    # ----------------
    inner_layer = layers.Activation('relu')(inner_layer)
    inner_layer = layers.SeparableConv2D(size, 3, padding='same')(inner_layer)
    inner_layer = layers.BatchNormalization()(inner_layer)
    # ----------------
    inner_layer = layers.MaxPooling2D(3, strides=2, padding='same')(inner_layer)
    residual = layers.Conv2D(
        size, 1, strides=2, padding='same')(previous_block_activation)           
    inner_layer = layers.add([inner_layer, residual])
    previous_block_activation = inner_layer
  # ----------------
  return inner_layer


def middle_layer(inner_layer, num_blocks=8):
  
  previous_block_activation = inner_layer

  for _ in range(num_blocks):
    inner_layer = layers.Activation('relu')(inner_layer)
    inner_layer = layers.SeparableConv2D(728, 3, padding='same')(inner_layer)
    inner_layer = layers.BatchNormalization()(inner_layer)
    # ----------------
    inner_layer = layers.Activation('relu')(inner_layer)
    inner_layer = layers.SeparableConv2D(728, 3, padding='same')(inner_layer)
    inner_layer = layers.BatchNormalization()(inner_layer)
    # ----------------
    inner_layer = layers.Activation('relu')(inner_layer)
    inner_layer = layers.SeparableConv2D(728, 3, padding='same')(inner_layer)
    inner_layer = layers.BatchNormalization()(inner_layer)
    # ----------------
    inner_layer = layers.add([inner_layer, previous_block_activation])
    previous_block_activation = inner_layer 
  # ----------------    
  return inner_layer
def exit_layer(inner_layer, num_classes):
  # ----------------
  previous_block_activation = inner_layer
  # ----------------
  inner_layer = layers.Activation('relu')(inner_layer)
  inner_layer = layers.SeparableConv2D(728, 3, padding='same')(inner_layer)
  inner_layer = layers.BatchNormalization()(inner_layer)
  # ----------------
  inner_layer = layers.Activation('relu')(inner_layer)
  inner_layer = layers.SeparableConv2D(1024, 3, padding='same')(inner_layer)
  inner_layer = layers.BatchNormalization()(inner_layer)
  # ----------------
  inner_layer = layers.MaxPooling2D(3, strides=2, padding='same')(inner_layer)
  # ----------------
  residual = layers.Conv2D(
      1024, 1, strides=2, padding='same')(previous_block_activation)
  inner_layer = layers.add([inner_layer, residual])
  # ----------------
  inner_layer = layers.SeparableConv2D(1536, 3, padding='same')(inner_layer)
  inner_layer = layers.BatchNormalization()(inner_layer)
  inner_layer = layers.Activation('relu')(inner_layer)
  # ----------------
  inner_layer = layers.SeparableConv2D(2048, 3, padding='same')(inner_layer)
  inner_layer = layers.BatchNormalization()(inner_layer)
  inner_layer = layers.Activation('relu')(inner_layer)
  # ----------------
  inner_layer = layers.GlobalAveragePooling2D()(inner_layer)
  return layers.Dense(num_classes, activation="softmax")(inner_layer)

keras.backend.clear_session()   # free keras memory

In [ ]:
input = keras.Input(shape=(48, 48, 1))
output = exit_layer(middle_layer(top_layer(input)), 7)
model = keras.Model(input, output)

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(learning_rate=0.003), metrics=['accuracy'])

In [ ]:
with tf.device("/GPU:0"):
  model_history = model.fit(train_data, train_lab, epochs=50, batch_size=1024, validation_data=(val_data, val_lab), verbose=1, callbacks=[tqdm_callback])

In [ ]:
history = model_history.history

In [ ]:
plt.plot(history["loss"], label="train loss")
plt.plot(history["val_loss"], label="val loss")
plt.legend()
plt.show()

In [ ]:
plt.plot(history['accuracy'], label="train acc")
plt.plot(history['val_accuracy'], label="val acc")
plt.legend()
plt.show()

In [ ]:
with tf.device("/GPU:0"):
  model.evaluate(test_data, test_lab, callbacks=[tqdm_callback])[1]